In [1]:
import pickle
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
import glob
from time import time

C:\Users\EXTREME_RED\AppData\Local\Temp\ipykernel_6692\4289264615.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [14]:
'''
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

filename = "Flickr8k.token.txt"
# load descriptions
doc = load_doc(filename)
print(doc[:300])
'''

1000268201_693b08cb0e.jpg#0	A child in a pink dress is climbing up a set of stairs in an entry way .
1000268201_693b08cb0e.jpg#1	A girl going into a wooden building .
1000268201_693b08cb0e.jpg#2	A little girl climbing into a wooden playhouse .
1000268201_693b08cb0e.jpg#3	A little girl climbing the s


In [27]:
import json


def read_json_file(json_file_path):
    descriptions = {}
    try:
        with open(json_file_path, 'r') as file:
            data = json.load(file)
            for item in data:
                if 'image_id' in item and 'paragraph' in item:
                    image_id_str = str(item['image_id'])
                    descriptions[image_id_str] = item['paragraph']
                else:
                    print("Invalid format: Missing 'image_id' or 'paragraph' key in JSON item.")
    except FileNotFoundError:
        print("File not found.")
    except json.JSONDecodeError:
        print("Invalid JSON format in file.")
    except Exception as e:
        print("An error occurred:", e)
    return descriptions


json_file_path = 'paragraphs_v1.json'  # Specify the path to your JSON file
descriptions = read_json_file(json_file_path)
#print("Descriptions:", descriptions)
print('Loaded: %d ' % len(descriptions))

Loaded: 19551 


In [28]:
list(descriptions.keys())[:5]

['2356347', '2317429', '2414610', '2365091', '2383120']

In [29]:
count = 0
for image_id, paragraph in descriptions.items():
    print(f"Image ID: {image_id}, Paragraph: {paragraph}")
    count += 1
    if count == 5:
        break

Image ID: 2356347, Paragraph: A large building with bars on the windows in front of it. There is people walking in front of the building. There is a street in front of the building with many cars on it. 
Image ID: 2317429, Paragraph: A white round plate is on a table with a plastic tablecloth on it. Two foil covered food halves are on the white plate along with a serving of golden yellow french fries. Next to the white plate in a short, topless, plastic container is a white sauce. Diagonal to the white plate are the edges of several other stacked plates. There are black shadows reflected on the table.
Image ID: 2414610, Paragraph: A woman in a blue tennis outfit stands on a green tennis court. She is swinging a blue tennis racket. There is a green tennis ball above her head. 
Image ID: 2365091, Paragraph: A large red and white train is traveling on tracks in a what looks to be a rural area. There are trees and hills in the background and the ground looks dry. The train has many large w

In [30]:
print(descriptions['2356347'])

A large building with bars on the windows in front of it. There is people walking in front of the building. There is a street in front of the building with many cars on it. 


In [32]:
import string

def clean_descriptions(descriptions):
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for key, desc in descriptions.items():
        # tokenize
        desc = desc.split()
        # convert to lower case
        desc = [word.lower() for word in desc]
        # remove punctuation from each token
        desc = [w.translate(table) for w in desc]
        # remove hanging 's' and 'a'
        desc = [word for word in desc if len(word)>1]
        # remove tokens with numbers in them
        desc = [word for word in desc if word.isalpha()]
        # store as string
        descriptions[key] = ' '.join(desc)

# clean descriptions
clean_descriptions(descriptions)

In [34]:
descriptions['2414610']

'woman in blue tennis outfit stands on green tennis court she is swinging blue tennis racket there is green tennis ball above her head'

In [37]:
def create_vocabulary(descriptions):
    vocabulary = set()
    for key, desc in descriptions.items():
        # Tokenize the paragraph into words
        words = desc.split()
        # Add unique words to the vocabulary set
        vocabulary.update(words)
    return vocabulary

# Create vocabulary
vocabulary = create_vocabulary(descriptions)
#print("Vocabulary:", vocabulary)
print("Vocabulary Size:", len(vocabulary))


Vocabulary Size: 13449


In [39]:
def save_descriptions_to_file(descriptions, file_path):
    with open(file_path, 'w') as file:
        for key, desc in descriptions.items():
            file.write(f"{key} {desc}\n")


file_path = 'descriptions.txt'  
save_descriptions_to_file(descriptions, file_path)


In [41]:
import json

def load_json_file(file_path):
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
        return data
    except FileNotFoundError:
        print("File not found.")
        return {}
    except json.JSONDecodeError:
        print("Invalid JSON format in file.")
        return {}
    except Exception as e:
        print("An error occurred:", e)
        return {}

def save_descriptions_to_file(descriptions, file_path):
    with open(file_path, 'w') as file:
        for key, desc in descriptions.items():
            file.write(f"{key} {desc}\n")

# Load train_images.json file
test_images_file_path = 'train_split.json'
with open(test_images_file_path, 'r') as file:
    test_images_data = json.load(file)

# Convert image IDs to strings
image_ids = [str(image_id) for image_id in test_images_data]

# Filter descriptions dictionary to include only image IDs present in train_images.json
filtered_descriptions = {key: desc for key, desc in descriptions.items() if key in image_ids}

# Save filtered descriptions to a text file
file_path = 'train_descriptions.txt'
save_descriptions_to_file(filtered_descriptions, file_path)
#print("Filtered descriptions saved to", file_path)



In [42]:
# Load test_images.json file
test_images_file_path = 'test_split.json'
with open(test_images_file_path, 'r') as file:
    test_images_data = json.load(file)

# Convert image IDs to strings
image_ids = [str(image_id) for image_id in test_images_data]

# Filter descriptions dictionary to include only image IDs present in test_images.json
filtered_descriptions = {key: desc for key, desc in descriptions.items() if key in image_ids}
print(len(filtered_descriptions))
# Save filtered descriptions to a text file
file_path = 'test_descriptions.txt'
save_descriptions_to_file(filtered_descriptions, file_path)

2489


In [43]:
# Load val_images.json file
test_images_file_path = 'val_split.json'
with open(test_images_file_path, 'r') as file:
    test_images_data = json.load(file)

# Convert image IDs to strings
image_ids = [str(image_id) for image_id in test_images_data]

# Filter descriptions dictionary to include only image IDs present in val_images.json
filtered_descriptions = {key: desc for key, desc in descriptions.items() if key in image_ids}
print(len(filtered_descriptions))
# Save filtered descriptions to a text file
file_path = 'val_descriptions.txt'
save_descriptions_to_file(filtered_descriptions, file_path)

2487
